# EnKF Experiments - GCS benchmark

## Imports

In [1]:
# Imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
from scipy.stats import shapiro, kruskal
import seaborn as sns
import sys

In [2]:
sys.path.append('../../../../stationsim/')
from ensemble_kalman_filter import EnsembleKalmanFilter, AgentIncluder
from stationsim_gcs_model import Model

In [3]:
sys.path.append('../')
from experiment_utils import Modeller, Processor, Visualiser

In [4]:
%matplotlib inline
np.random.seed(42)

## Functions

In [5]:
def tidy_dataframe(df, independent_col: str, dependent_cols: list):
    output = list()
    
    for i, row in df.iterrows():
        for col in dependent_cols:
            d = {independent_col: row[independent_col],
                 'variable':  col,
                 'value': row[col]}
            output.append(d)
    
    output = pd.DataFrame(output)
    return output

## Data paths

In [6]:
data_dir = '../results/data/baseline/'
model_dir = '../results/models/baseline/'
fig_dif = '../results/figures/baseline/'

## Experiment 0 - Baseline

### Run experiment

#### Benchmark

In [7]:
Modeller.run_enkf_benchmark()

../../../../stationsim/ensemble_kalman_filter.py:170: RuntimeWarning: EnKF received unexpected attribute (vanilla_ensemble_size).
  warns.warn(w, RuntimeWarning)


Running Ensemble Kalman Filter...
max_iterations:	None
ensemble_size:	None
assimilation_period:	None
filter_type:	EnsembleKalmanFilterType.STATE


#### Collisions

In [ ]:
Modeller.run_model_collisions()

  0%|          | 0/20 [00:00<?, ?it/s]

Running for 5 agents


  0%|          | 0/20 [00:00<?, ?it/s]

Running for 10 agents


  0%|          | 0/20 [00:00<?, ?it/s]

...

  0%|          | 0/20 [00:00<?, ?it/s]

Running for 25 agents


  0%|          | 0/20 [00:00<?, ?it/s]

Running for 30 agents


 20%|██        | 4/20 [00:37<02:06,  7.89s/it]

### Analysis

#### Benchmark

In [9]:
with open(model_dir + 'baseline.pkl', 'rb') as f:
    baseline_filter = pickle.load(f)

baseline_results = pd.DataFrame(baseline_filter.forecast_error)
baseline_results.head()

FileNotFoundError: [Errno 2] No such file or directory: '../results/models/baselinebaseline.pkl'

In [ ]:
plt.figure()
sns.lineplot(x='time', y='forecast', data=baseline_results)
plt.savefig(fig_dir + 'baseline_errors_10.pdf')
plt.show()

#### Collisions

In [ ]:
collisions = pd.read_csv(data_dir + 'model_collisions.csv')
collisions.head()

In [ ]:
if 'Unnamed: 0' in collisions.columns:
    collisions = collisions.drop('Unnamed: 0', axis=1)

In [ ]:
collisions.head()

In [ ]:
plt.figure()
sns.lineplot(x='population_size', y='collisions', data=collisions)
plt.savefig(fig_dir + 'baseline_collisions.pdf')
plt.show()